In [0]:
%python
import requests
import pandas as pd
import datetime
import json
import time

## A classe Collector no seu código é uma estrutura que encapsula a lógica de coleta, processamento e armazenamento de dados de uma API. Ela permite organizar o código de forma modular e reutilizável.

class Collector:
    
    def __init__(self, url, instance_name):
        self.url = url  # Define a URL base da API
        self.instance_name = instance_name  # Nome da instância coletora

    def get_content(self, **kwargs):
        resp = requests.get(self.url, params=kwargs)  # Faz a requisição GET na API, passando os parâmetros
        return resp  # Retorna a resposta da API

    def save_json(self, data):
        now = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")  # Gera um timestamp para nome do arquivo
        with open(f"/dbfs/mnt/jovemnerd/{now}.json", "w") as open_file:  # Define o caminho no DBFS e abre o arquivo para escrita
            json.dump(data, open_file)  # Salva os dados no formato JSON

    def save_parquet(self, data): 
        now = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")  # Gera um timestamp para nome do arquivo
        df = pd.DataFrame(data)  # Converte os dados para um DataFrame do Pandas
        df.to_parquet(f"/dbfs/mnt/jovemnerd/{now}.parquet", index=False)  # Salva os dados no formato Parquet

    def save_data(self, data, save_format):
        if save_format == 'json':  # Verifica o formato escolhido para salvar os dados
            self.save_json(data)  # Salva os dados como JSON
        elif save_format == 'parquet':  
            self.save_parquet(data)  # Salva os dados como Parquet
    
    def get_and_save(self, save_format='json', **kwargs):
        resp = self.get_content(**kwargs)  # Obtém os dados da API
        if resp.status_code == 200:  # Verifica se a requisição foi bem-sucedida
            if resp.content:  # Verifica se a resposta não está vazia
                data = resp.json()  # Converte a resposta para JSON
                self.save_data(data, save_format)  # Salva os dados no formato desejado
            else:
                data = None
                print("Resposta vazia da API")
        else:
            data = None  # Define data como None em caso de erro na requisição
            print(f"Request sem Sucesso: {resp.status_code}", resp.text)  # Exibe o código de erro e a resposta da API
        return data  # Retorna os dados obtidos ou None em caso de erro
    
    def auto_exec(self, save_format='json', date_stop='2000-01-01'):
        page = 1  # Inicializa a contagem de páginas
        while True:
            print(page)  # Exibe a página atual no console
            data = self.get_and_save(save_format=save_format,
                                     page=page,
                                     per_page=1000)  # Obtém e salva os dados da API

            if data is None:  # Se houver erro na requisição
                print("Erro ao coletar dados... Aguardando")  
                time.sleep(60 * 5)  # Aguarda 5 minutos antes de tentar novamente

            else:
                date_last = pd.to_datetime(data[-1]['date']["published_at"]).date()  # Converte a data do último registro para o formato de data
                if date_last < pd.to_datetime(date_stop).date():  # Verifica se a data do último registro é menor que a data de parada
                    break  # Encerra a coleta de dados
                elif len(data) < 1000:
                    break
                page += 1  # Incrementa a página para continuar a coleta
                time.sleep(5)  # Aguarda 5 segundos antes da próxima requisição

# Define a URL da API
url = "https://api.jovemnerd.com.br/wp-json/jovemnerd/v1/nerdcasts/"

# Cria uma instância do coletor para coletar episódios do Jovem Nerd
collect = Collector(url, "episodios")

# Executa a coleta automática de dados
collect.auto_exec()

In [0]:
dbutils.fs.mkdirs("dbfs:/mnt/jovemnerd")

In [0]:
dbutils.fs.ls("dbfs:/mnt/jovemnerd")

In [0]:
df = spark.read.json("dbfs:/mnt/jovemnerd/")

display(df)